### Word2Vec를 이용하기 위해 토크나이징

In [ ]:
import pandas as pd
import json
from konlpy.tag import Okt
import re

okt = Okt()

def tokenizer(line, okt, remove_stopwords = False, stop_words = []):
    # line: 전처리할 텍스트
    # okt: okt 객체를 반복적으로 생성하지 않고 미리 생성한 후 인자로 받는다.
    # remove_stopword: 불용어를 제거할지 여부 선택, 기본값은 False
    # stop_word: 불용어 사전은 사용자가 직접 입력해야 함. 기본값은 빈 리스트

    # okt 객체를 활용해 형태소 단위로 나눈다.
    word_token = okt.morphs(line, norm=True, stem=False)

    if remove_stopwords:
        # 불용어 제거(선택적)
        word_review = [token for token in word_token if not token in stop_words]
    
    return word_token

from string import ascii_letters
def char_3x_shift(st):
    # 한글 자음 리스트
    CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    # 한글 모음 리스트
    JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ']
    # 알파벳 리스트
    ALPHABET_LIST = list(ascii_letters)
    # 구두기호 리스트 (\ 제외)
    PUNCTUATION_LIST = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':',
                        ';', '<', '=', '>', '?', '@', '[', ']', '^', '_', '`', '{', '|', '}', '~']
    
    
    CHAR_LIST = [CHOSUNG_LIST, JUNGSUNG_LIST, ALPHABET_LIST]
    
    for l in CHAR_LIST:
        for c in l:
            st = re.sub(c+'{3,}', c*3, st)
            
    for p in PUNCTUATION_LIST:
        st = re.sub('\\'+p+'{3,}', p*3, st)
    
    return st

def strip_e(st):
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    return RE_EMOJI.sub(r'', st)

In [ ]:
# 채팅부분 토크나이징
import json

DATA_IN_PATH = './data/video_chat/'

train_data = pd.read_csv(DATA_IN_PATH + 'chat_log.csv')
tokenized_train_data = []

print('tokenizer start!')

batch_size = 10000
epoch_size = int(train_data.shape[0] / batch_size)
remainder = train_data.shape[0] % batch_size

for idx, line in enumerate(train_data['chat']):
    tokenized_train_data.append(tokenizer(strip_e(char_3x_shift(line)), okt))
    if idx % 10000 == 0:
        print('{} / {}'.format(idx, train_data.shape[0]))

print('tokenized_train_data_size: {}'.format(len(tokenized_train_data)))
print('done')

In [ ]:
# 채팅 앞뒤 3줄씩 합치기
list_size = len(tokenized_train_data[:-2])
for idx, item in enumerate(tokenized_train_data[:-2]):
    tokenized_train_data[idx] = tokenized_train_data[idx] + tokenized_train_data[idx+1] + tokenized_train_data[idx+2]
    if idx % 10000 == 0:
        print('{} / {}'.format(idx, list_size))
clean_train_data = tokenized_train_data[:-2]
print(clean_train_data[:5])

In [ ]:
len(clean_train_data)

In [ ]:
# 토큰화된 채팅 json 형식으로 저장
DATA_IN_PATH = './data/video_chat/'
with open(DATA_IN_PATH + 'chat_token.json',
          'w', encoding='utf-8') as file:
    try:
        json.dump(clean_train_data, file, ensure_ascii=False, indent="\t")
    except Exception as e:
        print(e)

### Word2vc 벡터화

In [ ]:
# 토큰화된 채팅 파일 가져오기
import json
DATA_IN_PATH = './data/video_chat/'
with open(DATA_IN_PATH + 'chat_token.json', 'r', encoding='utf-8') as file:
    clean_train_data = json.load(file)

In [ ]:
len(clean_train_data)

In [ ]:
# 학습 시 필요한 하이퍼 파라미터
num_features = 150  # 워드 벡터 특징값 수
min_word_count = 30  # 단어에 대한 최소 빈도 수
num_workers = 4  # 프로세스 개수
context = 5  # 컨텍스트 윈도우 크기
downsampling = 1e-3  # 다운 샘플링 비율
sg = 1  # 스킵그램 사용 여부
hs = 0  # 0: negative sampling, 1: hierarchical softmax
negative = 5  # negative sampling 할 노이즈 단어의 수
#batch_words_size = 30  # 배치 단어의 수

In [ ]:
# word2vec 학습 진행상황을 확인할 수 있도록 logging 설정
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

In [ ]:
# word2vec 학습
from gensim.models import word2vec
print('Training model...')
model = word2vec.Word2Vec(clean_train_data,
                         workers=num_workers,
                         size=num_features,
                         min_count=min_word_count,
                         window=context,
                         sample=downsampling,
                         sg = sg,
                         hs = hs,
                         negative = negative
                         #batch_words=batch_words_size
                         )

In [ ]:
# 모델의 하이퍼파라미터를 설정한 내용을 모델 이름에 담는다면 나중에 참고하기 좋을 것이다.
# 모델을 저장하면 Word2Vec.load()를 통해 모델을 다시 사용할 수 있다.
model_name = '150features_30minwords_5context'
model.save(model_name)

In [ ]:
# 모델 불러오기
from gensim.models import word2vec
model_name = '150features_30minwords_5context'
model = word2vec.Word2Vec.load(model_name)

### word2vec 벡터 연산

In [ ]:
print(model.most_similar(positive=["ㅋㅋㅋ"], topn=100))